In [1]:
import plotly.express as px
import pandas as pd 
import numpy as np
from tqdm.notebook import tqdm
import datetime, pickle
from sklearn.utils import shuffle
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from itertools import product
import itertools, random, os
import plotly.express as px
from operator import itemgetter
import math

pd.set_option('max_column', None)

In [2]:
DIR_PATH = f'simulations/days_montecarlo_simulations/'

In [3]:
# Get min max spread in day simulations
dict_simulazioni = {}
min_spread = 999999999
max_spread = -99999999
for file in os.listdir(os.fsencode(DIR_PATH)):
     filename = os.fsdecode(file)
     if filename.endswith(".pkl"):
        current_date = datetime.strptime(filename.replace('.pkl',''), '%Y%m%d')

        with open(os.path.join(DIR_PATH, filename), 'rb') as f:
            temp_dict = pickle.load(f)

        spread_values = np.array([v[2] for v in temp_dict.values()])
        dict_simulazioni[current_date] = spread_values

        if min(spread_values) < min_spread:
            min_spread = min(spread_values)

        if max(spread_values) > max_spread:
            max_spread = max(spread_values)

In [4]:
# I must look for the optimum point that has been highest and most frequent at the same time in the past 
NUMERO_VICINI = 30

array_spread_value = []
array_sharpe_ratio = []
for current_spread in np.arange(min_spread, max_spread, 0.001):
    date_spread_values = []
    for current_date in dict_simulazioni:
        temp_spread_array = dict_simulazioni[current_date]
        nearest_spreads_list = temp_spread_array[np.argpartition(abs(temp_spread_array - current_spread), NUMERO_VICINI)[:NUMERO_VICINI]] # average the spread of the closest N elements
        date_spread_values.append(nearest_spreads_list.mean())

    sharpe_ratio = np.array(date_spread_values).mean() / np.array(date_spread_values).std()

    array_spread_value.append(current_spread)
    array_sharpe_ratio.append(sharpe_ratio)
    
    #break

In [62]:
# Identifying the optimal spread over time
print(f"Max sharpe ratio: {max(array_sharpe_ratio)} at spread: {array_spread_value[array_sharpe_ratio.index(max(array_sharpe_ratio))]}")
fig = px.line(x=array_spread_value, y=array_sharpe_ratio)
fig.show()

Max sharpe ratio: 4772.221199129733 at spread: 0.3941853716597983
